In [2]:
!pip install pandas sklearn numerapi

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=2479d116515d1c0209cadacbf8c1271406ba91372fcbd2a303b76dd0e826e241
  Stored in directory: /Users/A.R/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [78]:
import numpy as np
import pandas as pd
import numerapi
import sklearn

In [13]:
from sklearn.model_selection import cross_validate

from sklearn.model_selection import GridSearchCV as GS

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier as RFC

from sklearn.metrics import accuracy_score, log_loss

In [4]:
# download the latest training dataset (takes around 30s)
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
training_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,...,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.50
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,...,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,...,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.25
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,...,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.25
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,...,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75


In [67]:
# download the latest tournament dataset (takes around 30s)
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
tournament_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,...,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.25
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,...,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.50
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,...,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,...,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.50
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,...,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,0.75


In [69]:
#le encoding since RFC cannot deal with categorical variables
from sklearn import preprocessing
    
le = preprocessing.LabelEncoder()
for column_name in training_data.columns:
    if training_data[column_name].dtype == object:
        training_data[column_name] = le.fit_transform(training_data[column_name])
    else:
        pass
for column_name in training_data.columns:
    if tournament_data[column_name].dtype == object:
        tournament_data[column_name] = le.fit_transform(tournament_data[column_name])
    else:
        pass

In [71]:
#Data transformation into integer to be accepted by the RFC

from sklearn import utils

print(utils.multiclass.type_of_target(training_data))
print(utils.multiclass.type_of_target(training_data.astype('int')))

#Therefore
training_data_int = training_data.astype('int')

#Similarly for tournament data, but first fill NA by 0
tournament_data = tournament_data.fillna(0)
tournament_data_int = tournament_data.astype('int')

In [53]:
# splitting my arrays in ratio of 30:70 percent
features_train, features_test, labels_train, labels_test = train_test_split(training_data_int.iloc[:,:21], training_data_int['target'], test_size=0.3, random_state=0)

# parameters
parameters = {
        'n_estimators': [ 20,25 ],
        'random_state': [ 0 ],
        'max_features': [ 2 ],
        'min_samples_leaf': [120,220,300]
}


In [54]:
# implementing my classifier
clf = RFC(n_estimators=25, random_state=0).fit(features_train, labels_train)

In [58]:
#To Calculate the logloss and accuracy of the model, but first find

prob_predictions_class_test = clf.predict(features_test)
prob_predictions_test = clf.predict_proba(features_test)
print(prob_predictions_class_test)
print(prob_predictions_test)

[0 0 0 ... 0 0 0]
[[1.  0. ]
 [1.  0. ]
 [1.  0. ]
 ...
 [1.  0. ]
 [1.  0. ]
 [0.8 0.2]]


In [57]:
#Metric to see how good the model is

logloss = log_loss(labels_test,prob_predictions_test)
accuracy = accuracy_score(labels_test, prob_predictions_class_test, normalize=True,sample_weight=None)
print ('accuracy', accuracy)
print ('logloss', logloss)

accuracy 0.9442684150043509
logloss 0.8500440199702978


In [88]:
# predict class probabilities for the tourney set

prob_predictions_tourney = clf.predict_proba(tournament_data.iloc[:,1:22])
print(prob_predictions_tourney)

[[0.72 0.28]
 [1.   0.  ]
 [0.76 0.24]
 ...
 [0.92 0.08]
 [0.92 0.08]
 [0.76 0.24]]


In [87]:
# extract the probability of being in a class 1
probability_class_of_one = np.array([x[1] for x in prob_predictions_tourney[:]]) # List comprehension
print(probability_class_of_one)

[0.28 0.   0.24 ... 0.08 0.08 0.24]


In [92]:
t_id = tournament_data['id']

np.savetxt(
    'Desktop/surfer366/4_Work_HUB/Numerai_CHALLENGE/probability.csv',    # file name
    probability_class_of_one,# array to save
    fmt='%.2f',              # formatting, 2 digits in this case
    delimiter=',',           # column delimiter
    newline='\n',            # new line character
    header= 'probability')   # file header

np.savetxt(
    'Desktop/surfer366/4_Work_HUB/Numerai_CHALLENGE/t_id.csv',          # file name
    t_id,                   # array to save
    fmt='%.d',              # formatting, 2 digits in this case
    delimiter=',',          # column delimiter
    newline='\n',           # new line character
    header= 't_id')   # file header